In [1]:
# libraries
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import scale

letters = pd.read_csv("letter-recognition.csv")

In [2]:
print("Dimensions: ", letters.shape, "\n")

Dimensions:  (20000, 17) 



In [3]:
letters.columns = ['letter', 'xbox', 'ybox', 'width', 'height', 'onpix', 'xbar', 'ybar', 'x2bar', 'y2bar', 'xybar', 'x2ybar', 'xy2bar', 'xedge', 'xedgey', 'yedge', 'yedgex']
print(letters.columns)

Index(['letter', 'xbox', 'ybox', 'width', 'height', 'onpix', 'xbar', 'ybar',
       'x2bar', 'y2bar', 'xybar', 'x2ybar', 'xy2bar', 'xedge', 'xedgey',
       'yedge', 'yedgex'],
      dtype='object')


In [4]:
order = list(np.sort(letters['letter'].unique()))
print(order)

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']


In [5]:
round(letters.drop('letter', axis=1).mean(), 2)

xbox      4.02
ybox      7.04
width     5.12
height    5.37
onpix     3.51
xbar      6.90
ybar      7.50
x2bar     4.63
y2bar     5.18
xybar     8.28
x2ybar    6.45
xy2bar    7.93
xedge     3.05
xedgey    8.34
yedge     3.69
yedgex    7.80
dtype: float64

In [6]:
X = letters.drop("letter", axis = 1)
y = letters['letter']

In [7]:
X_scaled = scale(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size = 0.3, random_state = 101)

In [8]:
print(y_train)

10593    H
18903    N
6491     X
17136    V
19651    Z
        ..
5695     K
8006     P
17745    P
17931    U
13151    M
Name: letter, Length: 14000, dtype: object


In [9]:
import random
print(type(X_train))
print((y_train.to_numpy()))

<class 'numpy.ndarray'>
['H' 'N' 'X' ... 'P' 'U' 'M']


In [10]:
import math
def markov_samp(X_train, Y_train, k = 10, q = 1.2):
    samp = []
    for ctr in range(k):
        # Init_sampl = np.concatenate((X_train, np.array()), axis = 0)
        Init_sampl = np.concatenate((np.vstack(X_train), np.vstack(Y_train.to_numpy())), axis = 1)
        Dtr = random.sample(list(Init_sampl), int(2000))

        m = len(Dtr)
        print("M : " , m)
        # mplus =np.count_nonzero(np.array(Dtr[16]) == 'A')
        # mplus = Dtr[16].count('A')
        mneg = 0
        mplus = 0

        index = np.random.choice(len(Dtr), 1, replace=False)  
        Dtra = np.array(Dtr)
        print(type(Dtra))
        zt = Dtra[index][0]

        # print("zt : ", zt)
        if m%2 == 0:
            if zt[16] == 'A':
                mplus += 1;
            else:
                mneg += 1
        

        model_linear = SVC(kernel='linear')#
        model_linear.fit(X_train, Y_train)#
        
        yzt = model_linear.predict(np.array(zt)[0:16].reshape(1, -1))
        fxy1 = 1
        if not yzt==zt[16]:
            fxy1 = 2 
        lzt = math.exp(0-fxy1)

        Pd = 0
        Pdd = 0
        
        while(mplus + mneg < m ):
            zstar = Dtra[np.random.choice(len(Dtr), 1, replace=False)][0]
            ystar = model_linear.predict(np.array(zstar)[ 0:16].reshape(1, -1)) #
            fxy = 1
            if not ystar==zstar[16]:
                fxy = 2 
            lzstar = math.exp(0-fxy)
            P = math.exp(lzt-lzstar)
            # print(zt)
            yt = zt[16]
            zt = zstar

            if P == 1:
                if zt[16] == yt:
                    r = random.uniform(0.001, 1.0)
                    if r <= P:
                        samp.append(zstar) #Pdash = e−y∗f0 /e−ytf0
                else:
                    ct = 1 if yt=='A' else -1
                    cst = 1 if ystar=='A' else -1
                    Pd = math.exp(ct*fxy1-cst*fxy)
                    r = random.uniform(0.001, 1.0)
                    if r <= Pd:
                        samp.append(zstar) #P

            if len(samp) == k:
                Pdd = q*P
                samp.append(zstar) #Pdd
                
            ztp1 = zstar
            if yt == 'A':
                mplus += 1
            else:
                mneg += 1

            if P > 1 or Pd > 1 or Pdd > 1:
                samp.append(zstar)
    return samp


In [11]:
nsamp = np.array(markov_samp(X_train, y_train))
print(nsamp.shape)
X_train = nsamp[:, 0:16]
y_train = nsamp[:, 16]
print(X_train.shape)
print(y_train.shape)

M :  2000
<class 'numpy.ndarray'>
M :  2000
<class 'numpy.ndarray'>
M :  2000
<class 'numpy.ndarray'>
M :  2000
<class 'numpy.ndarray'>
M :  2000
<class 'numpy.ndarray'>
M :  2000
<class 'numpy.ndarray'>
M :  2000
<class 'numpy.ndarray'>
M :  2000
<class 'numpy.ndarray'>
M :  2000
<class 'numpy.ndarray'>
M :  2000
<class 'numpy.ndarray'>
(20009, 17)
(20009, 16)
(20009,)


In [12]:
# print(y_train)
Y_train = []
for i in y_train:
    # print(i)
    Y_train.append(ord(i))
# print(Y_train)

In [13]:
print(X_test.shape)

(6000, 16)


In [14]:
model_linear = SVC(kernel='linear')
model_linear.fit(X_train, Y_train)
y_pred = model_linear.predict(X_test)


In [15]:
# print(y_pred)
Y_test = []
for i in y_test:
    Y_test.append(ord(i))
# print(Y_test)

In [16]:
print("Accuracy linear kernel : ", metrics.accuracy_score(y_true=Y_test, y_pred=y_pred), "\n")
print(metrics.confusion_matrix(y_true=Y_test, y_pred=y_pred))

Accuracy linear kernel :  0.8363333333333334 

[[184   0   0   0   0   0   1   1   0   0   1   2   1   2   0   0   1   3
    4   1   2   0   0   0   3   3]
 [  0 187   0   3   1   0   4   7   0   0   0   0   0   1   0   1   0   8
    3   0   0   1   0   1   1   0]
 [  2   0 202   0   5   0  12   1   0   0   3   0   1   0   3   0   0   0
    0   0   1   0   0   0   0   0]
 [  1  17   0 207   0   1   4   2   1   0   0   0   1   3   2   0   1   6
    0   1   0   0   0   0   0   1]
 [  0   1   4   0 201   2   4   3   0   0   2   5   0   0   0   0   2   1
    1   5   0   0   0   1   0   2]
 [  4   0   2   2   0 192   1   2   1   1   0   0   0   1   0   3   0   0
    2   8   0   0   0   1   5   0]
 [  0   1  12   3  10   2 154   0   0   0   4   2   2   0   2   0  13   1
    8   0   0   2   3   0   0   0]
 [  3   2   3  12   0   4   3 140   1   1   5   1   2   1  17   0   2  11
    0   0   4   1   0   2   2   0]
 [  0   0   1   3   0   7   1   0 185   9   0   1   0   0   1   0   0   0
    0  

In [17]:
non_linear_model = SVC(kernel='rbf')
non_linear_model.fit(X_train, Y_train)
y_pred = non_linear_model.predict(X_test)

In [18]:
print("Accuracy rbf kernel:", metrics.accuracy_score(y_true=Y_test, y_pred=y_pred), "\n")
print(metrics.confusion_matrix(y_true=Y_test, y_pred=y_pred))

Accuracy rbf kernel: 0.935 

[[195   0   0   1   0   0   1   0   0   0   0   1   1   0   0   0   0   0
    0   0   2   1   0   0   3   4]
 [  0 207   0   1   1   0   2   1   0   0   0   0   0   0   0   0   0   4
    1   0   0   0   0   1   0   0]
 [  0   0 215   0   3   0   7   1   0   0   0   0   0   0   4   0   0   0
    0   0   0   0   0   0   0   0]
 [  0   8   0 231   0   0   1   3   0   0   0   0   0   0   1   0   0   1
    1   0   1   0   0   0   0   1]
 [  0   0   3   0 223   1   4   0   0   0   0   0   0   0   0   0   0   1
    1   0   0   0   0   0   0   1]
 [  0   1   0   1   0 209   0   4   1   0   0   0   0   1   0   4   0   0
    1   3   0   0   0   0   0   0]
 [  0   0   4   4   2   1 202   0   0   0   0   1   1   0   0   0   0   1
    0   0   0   1   2   0   0   0]
 [  0   2   2   8   0   1   4 176   0   0   1   0   1   1   4   1   3  11
    0   0   2   0   0   0   0   0]
 [  0   0   1   0   0   2   0   0 194  13   0   0   0   0   0   3   0   0
    1   0   0   0   0   1

In [19]:
non_linear_model = SVC(kernel='poly')
non_linear_model.fit(X_train, Y_train)
y_pred = non_linear_model.predict(X_test)

In [20]:
print("Accuracy polynomial kernel:", metrics.accuracy_score(y_true=Y_test, y_pred=y_pred), "\n")
print(metrics.confusion_matrix(y_true=Y_test, y_pred=y_pred))

Accuracy polynomial kernel: 0.8911666666666667 

[[191   0   0   1   0   0   0   0   0   1   0   1   0   2   2   0   0   1
    0   0   1   0   0   2   2   5]
 [  0 184   0   3   1   0   2   5   0   0   0   0   1   0  14   0   0   2
    1   0   0   2   0   3   0   0]
 [  0   0 204   0   4   0   7   0   0   0   1   0   0   0  12   0   0   0
    0   0   0   0   0   2   0   0]
 [  0   4   0 222   0   0   1   4   1   0   0   0   0   1  13   0   0   2
    0   0   0   0   0   0   0   0]
 [  0   0   1   0 206   0   6   1   0   0   0   0   0   0   2   0   3   0
    0   0   0   0   0  15   0   0]
 [  0   0   0   3   0 202   0   1   3   0   0   0   0   1   2   5   0   0
    1   2   0   0   0   5   0   0]
 [  0   0   1   3   1   0 186   0   0   0   2   1   2   0  20   0   1   0
    0   0   0   0   1   0   1   0]
 [  1   1   1   7   0   0   4 136   0   3   1   0   0   1  38   0   1   4
    0   0   0   0   0  18   1   0]
 [  0   0   1   0   0   2   0   0 195   5   0   0   0   0   0   2   0   0
    2